In [ ]:
from evaluate import main
from hydra import compose, initialize
from omegaconf import OmegaConf

initialize(config_path="config")


In [ ]:

cfg = compose(config_name="evaluate", overrides=[
    "carla_sh_path=/home/carla/CarlaUE4.sh", "agent.mile.ckpt=/home/carla/mile.ckpt", "port=2000"])
main(cfg)